In [19]:
from selenium.common.exceptions import NoSuchElementException
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import pandas as pd
import re
from selenium.webdriver.common.keys import Keys
import time
import datetime
from IPython.display import clear_output
from random import randint

In [20]:
# selenium 4
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.implicitly_wait(15)

In [9]:
# selenium 4
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from webdriver_manager.microsoft import EdgeChromiumDriverManager

driver = webdriver.Edge(service=EdgeService(EdgeChromiumDriverManager().install()))

In [21]:
container = pd.DataFrame(columns= ['Products', 'Prices', 'URL', 'Sales'])
search_key_list = ['HP%20惠普', 'Acer%20宏碁', 'DELL%20戴爾', 'Lenovo']
brandNo_list = ['20160808155739954', '20160808155608760', '20160808155655766', '20160808155811738']
number =  3

In [22]:
search_key = search_key_list[number]
brandNo = brandNo_list[number]
driver.get("https://www.momoshop.com.tw/main/Main.jsp")
brand_url = 'https://www.momoshop.com.tw/search/searchShop.jsp?keyword=' + search_key + '&searchType=1&curPage=1&_isFuzzy=0&brand=' + search_key + '&brandNo='+ brandNo +'&showType=chessboardType&isBrandCategory=N&serviceCode=MT01'
# 獲取當前網頁的URL
driver.get(brand_url)
current_url = driver.current_url

# 找到"curPage="的位置
start_index = current_url.find("curPage=")

if start_index != -1:
    # 找到"curPage="的結尾位置（即數字的結尾位置）
    end_index = current_url.find("&", start_index)
    
    # 提取curPage的數字部分
    cur_page_number = current_url[start_index + len("curPage="):end_index]
    
    # 將curPage的數字替換為{}
    base_url = current_url[:start_index] + "curPage={}" + current_url[end_index:]
    
    # 打印提取的curPage數字
    print("提取的curPage數字是:", cur_page_number)
    
    # 打印修改後的URL
    print("修改後的URL是:", base_url)

final_page_index = int(driver.find_element(By.XPATH, '//*[@id="BodyBase"]//div[@class="pageArea"]/dl[1]/dt[1]/span[2]').get_attribute('textContent').split('/')[1])
print(search_key,"的搜尋結果共有:", final_page_index, '頁')

Prods_prices = pd.DataFrame(columns=['ProdNames', 'Prices', 'SloganTitle'])

for page_idx in range(1, final_page_index + 1):
    # 根據page_idx生成完整的URL
    url = base_url.format(page_idx)
    
    # 訪問該頁面
    driver.get(url)
    driver.implicitly_wait(randint(5, 10))
    prices = driver.find_elements(By.CLASS_NAME, 'price')
    prdnames = driver.find_elements(By.CLASS_NAME, 'prdName')
    slogans = driver.find_elements(By.CLASS_NAME, 'sloganTitle')

    for idx in range(len(prices)):
        price_text = prices[idx].text
        prd_text = prdnames[idx].text
        slogan = slogans[idx].get_attribute('textContent')
        new_row = pd.Series({'ProdNames': prd_text, 'Prices': price_text, 'SloganTitle':slogan}).to_frame().T
    
        # 将抓取到的信息添加到 DataFrame
        Prods_prices = pd.concat([Prods_prices, new_row], ignore_index=True)

提取的curPage數字是: 1
修改後的URL是: https://www.momoshop.com.tw/search/searchShop.jsp?keyword=Lenovo&searchType=1&curPage={}&_isFuzzy=0&brand=Lenovo&brandNo=20160808155811738&showType=chessboardType&isBrandCategory=N&serviceCode=MT04
Lenovo 的搜尋結果共有: 18 頁


KeyboardInterrupt: 

In [8]:
collect_date = pd.DataFrame({"date": [datetime.date.today()] * Prods_prices.shape[0]})
# Concatenate xxx and result_df along columns axis (axis=1)
Prods_prices = pd.concat([collect_date, Prods_prices], axis=1)
Prods_prices

,date,ProdNames,Prices,SloganTitle
0,2023-12-14,【Lenovo】15.6吋N4120輕薄筆電(IdeaPad Slim 1/82V7005B...,"$7,990",滿1件折5000
1,2023-12-14,【Lenovo】升級16G記憶體★15.6吋i5輕薄筆電(IdeaPad Slim 3i/8...,"$15,111",滿1件折3879
2,2023-12-14,【Lenovo】15.6吋i5輕薄筆電(IdeaPad Slim 3i/82RK00QWTW...,"$15,111",滿1件折2879
3,2023-12-14,【Lenovo】15.6吋R5輕薄筆電(IdeaPad Slim 1/82R400F8TW/...,"$11,990",滿1件折2500
4,2023-12-14,【Lenovo】15.6吋i5 RTX2050電競筆電(LOQ/i5-12450H/8G/5...,"$21,990",滿1件折4000
...,...,...,...,...
525,2023-12-14,【Lenovo】HR17 9.3吋聯想雙鏡頭流媒體後視鏡觸控屏行車記錄器,"$1,690",螢幕外框鏡面範圍達9.3吋
526,2023-12-14,【Lenovo】Tab M8 LTE 3G/32G 8吋平板(TB-8506X),"$3,980",
527,2023-12-14,【Lenovo】LP40 無線耳機,$580,智能觸控
528,2023-12-14,【Lenovo】Lenovo C2 聯想鑰匙扣型錄音筆16G,"$1,272",滿1件享8折


In [11]:
search_result = Prods_prices.copy()
search_result

,date,ProdNames,Prices,SloganTitle
0,2023-12-14,【Lenovo】15.6吋N4120輕薄筆電(IdeaPad Slim 1/82V7005B...,"$7,990",滿1件折5000
1,2023-12-14,【Lenovo】升級16G記憶體★15.6吋i5輕薄筆電(IdeaPad Slim 3i/8...,"$15,111",滿1件折3879
2,2023-12-14,【Lenovo】15.6吋i5輕薄筆電(IdeaPad Slim 3i/82RK00QWTW...,"$15,111",滿1件折2879
3,2023-12-14,【Lenovo】15.6吋R5輕薄筆電(IdeaPad Slim 1/82R400F8TW/...,"$11,990",滿1件折2500
4,2023-12-14,【Lenovo】15.6吋i5 RTX2050電競筆電(LOQ/i5-12450H/8G/5...,"$21,990",滿1件折4000
...,...,...,...,...
525,2023-12-14,【Lenovo】HR17 9.3吋聯想雙鏡頭流媒體後視鏡觸控屏行車記錄器,"$1,690",螢幕外框鏡面範圍達9.3吋
526,2023-12-14,【Lenovo】Tab M8 LTE 3G/32G 8吋平板(TB-8506X),"$3,980",
527,2023-12-14,【Lenovo】LP40 無線耳機,$580,智能觸控
528,2023-12-14,【Lenovo】Lenovo C2 聯想鑰匙扣型錄音筆16G,"$1,272",滿1件享8折


# brand

In [4]:
def find_next_page():
    try:
        button = driver.find_element(By.XPATH, '//*[@id="goodsAttrRoot"]/div[3]/div[5]/dl/dd/a')
        next_page = button.get_attribute('textContent')
    except NoSuchElementException:
        button = None
        next_page = None
    return button, next_page

def click_next_page(button, next_page):
    if (button is not None) and (next_page == '下一頁') :
        button.click()
    else:
        print('沒有下一頁')

In [26]:
driver.get("https://www.momoshop.com.tw/main/Main.jsp")

In [27]:
driver.get('https://www.momoshop.com.tw/category/LgrpCategory.jsp?l_code=2142800000')

In [25]:

left_column = driver.find_elements(By.XPATH, '//*[@id="bt_category_Content"]/ul/li/a')
href_list = [element.get_attribute('href') for element in left_column][1:]
brand_df = pd.DataFrame(columns = ['product Name', 'Product URL', 'Price', 'special_title'])
df_idx = -1
for ref in href_list:
    driver.get(ref)
    driver.refresh()
    time.sleep(3)
    try:
        total_page = driver.find_element(By.XPATH, '//*[@id="goodsAttrRoot"]/div[3]/div[5]/dl/dt/span').get_attribute('innerText').split('/')[1]
    except NoSuchElementException:
        total_page = 1
    for p in range(int(total_page)):
        if p !=0:
            click_next_page(*find_next_page())
            time.sleep(3)
        items = driver.find_elements(By.XPATH, '//*[@class="prdListArea bt770class"]/ul/li')
        for j in items:
            df_idx += 1
            special_title = j.find_element(By.XPATH, './/span[@class="specialTitle"]').text
            prodName = j.find_element(By.XPATH, './/h3[@class="prdName"]').text
            prodURL = j.find_element(By.XPATH, './/a[@class="prdUrl"]').get_attribute('href')
            price = j.find_element(By.XPATH, './/span[@class="prdPrice"]').text
            row = [prodName, prodURL, price, special_title]
            brand_df.loc[df_idx] = row

KeyboardInterrupt: 

In [14]:
brand_df

,product Name,Product URL,Price,special_title
0,【Lenovo】15.6吋N4120輕薄筆電(IdeaPad Slim 1/82V7005B...,https://www.momoshop.com.tw/goods/GoodsDetail....,"$7,990(售價已折)",滿1件折5000
1,【Lenovo】15.6吋R5輕薄筆電(IdeaPad Slim 1/82R400F8TW/...,https://www.momoshop.com.tw/goods/GoodsDetail....,"$11,990(售價已折)",滿1件折2500
2,【Lenovo】升級16G記憶體★15.6吋i5輕薄筆電(IdeaPad Slim 3i/8...,https://www.momoshop.com.tw/goods/GoodsDetail....,"$15,111(售價已折)",滿1件折3879
3,【Lenovo】14吋i7輕薄商用筆電(S14 G3/82TWA00FTW/i7-1255U...,https://www.momoshop.com.tw/goods/GoodsDetail....,"$19,912(售價已折)",滿1件折5078
4,【Lenovo】升級16G記憶體★15.6吋i5輕薄筆電(IdeaPad Slim 3i/8...,https://www.momoshop.com.tw/goods/GoodsDetail....,"$15,611(售價已折)",滿1件折3379
...,...,...,...,...
196,【ThinkPad 聯想】14吋i5商用筆電(E14/i5-1340P/8G/512G/W11H),https://www.momoshop.com.tw/goods/GoodsDetail....,"$23,900(售價已折)",滿1件折2090
197,【ThinkPad 聯想】福利品 14吋i5商務筆電(E14/i5-1240P/8G/512...,https://www.momoshop.com.tw/goods/GoodsDetail....,"$21,900",超值福利品!!
198,【ThinkPad 聯想】+8G記憶體組★14吋i5商用筆電(E14/i5-1340P/8G...,https://www.momoshop.com.tw/goods/GoodsDetail....,"$26,490(售價已折)",滿1件折500
199,【ThinkPad 聯想】福利品 15.6吋i7商務筆電(E15/i7-1260P/8G/5...,https://www.momoshop.com.tw/goods/GoodsDetail....,"$23,900(售價已折)",滿1件折1000


In [15]:
collect_date = pd.DataFrame({"date": [datetime.date.today()] * brand_df.shape[0]})
# Concatenate xxx and result_df along columns axis (axis=1)
brand_df = pd.concat([collect_date, brand_df], axis=1)
brand_df

,date,product Name,Product URL,Price,special_title
0,2023-12-14,【Lenovo】15.6吋N4120輕薄筆電(IdeaPad Slim 1/82V7005B...,https://www.momoshop.com.tw/goods/GoodsDetail....,"$7,990(售價已折)",滿1件折5000
1,2023-12-14,【Lenovo】15.6吋R5輕薄筆電(IdeaPad Slim 1/82R400F8TW/...,https://www.momoshop.com.tw/goods/GoodsDetail....,"$11,990(售價已折)",滿1件折2500
2,2023-12-14,【Lenovo】升級16G記憶體★15.6吋i5輕薄筆電(IdeaPad Slim 3i/8...,https://www.momoshop.com.tw/goods/GoodsDetail....,"$15,111(售價已折)",滿1件折3879
3,2023-12-14,【Lenovo】14吋i7輕薄商用筆電(S14 G3/82TWA00FTW/i7-1255U...,https://www.momoshop.com.tw/goods/GoodsDetail....,"$19,912(售價已折)",滿1件折5078
4,2023-12-14,【Lenovo】升級16G記憶體★15.6吋i5輕薄筆電(IdeaPad Slim 3i/8...,https://www.momoshop.com.tw/goods/GoodsDetail....,"$15,611(售價已折)",滿1件折3379
...,...,...,...,...,...
196,2023-12-14,【ThinkPad 聯想】14吋i5商用筆電(E14/i5-1340P/8G/512G/W11H),https://www.momoshop.com.tw/goods/GoodsDetail....,"$23,900(售價已折)",滿1件折2090
197,2023-12-14,【ThinkPad 聯想】福利品 14吋i5商務筆電(E14/i5-1240P/8G/512...,https://www.momoshop.com.tw/goods/GoodsDetail....,"$21,900",超值福利品!!
198,2023-12-14,【ThinkPad 聯想】+8G記憶體組★14吋i5商用筆電(E14/i5-1340P/8G...,https://www.momoshop.com.tw/goods/GoodsDetail....,"$26,490(售價已折)",滿1件折500
199,2023-12-14,【ThinkPad 聯想】福利品 15.6吋i7商務筆電(E15/i7-1260P/8G/5...,https://www.momoshop.com.tw/goods/GoodsDetail....,"$23,900(售價已折)",滿1件折1000


In [16]:
current_date = datetime.date.today().strftime('%Y%m%d')
brand_df.to_csv(f'brand_df_{current_date}.csv', index=False)

In [17]:
from glob import glob
file_pattern = 'brand_df_*.csv'
file_list = glob(file_pattern)

merged_df = pd.read_csv('brand_df_20231208.csv')[['product Name', 'Product URL']]

# 透過迴圈讀取並串接檔案
for file in file_list:
    df = pd.read_csv(file)
    merged_df = pd.merge(merged_df, df, how='outer', on=['product Name', 'Product URL'], suffixes=('', '_'+file[-12:-4]))

# 儲存最終結果
merged_df.to_csv('merged_brand_df.csv', index=False)

In [18]:
merged_df

,product Name,Product URL,date,Price,special_title,date_20231212,Price_20231212,special_title_20231212,date_20231214,Price_20231214,special_title_20231214
0,【Lenovo】15.6吋i5輕薄筆電(IdeaPad Slim 3/83EM0008TW/...,https://www.momoshop.com.tw/goods/GoodsDetail....,2023-12-08,"$18,990",熱銷i5效能，窄邊大螢幕,2023-12-12,"$18,990",熱銷i5效能，窄邊大螢幕,2023-12-14,"$18,990",熱銷i5效能，窄邊大螢幕
1,【Lenovo】15.6吋i5 RTX3050Ti電競筆電(Legion 5/i5-1250...,https://www.momoshop.com.tw/goods/GoodsDetail....,2023-12-08,"$24,990(售價已折)",滿1件折12500,2023-12-12,"$24,990(售價已折)",滿1件折12500,2023-12-14,"$24,990(售價已折)",滿1件折12500
2,【Lenovo】14吋i7輕薄筆電(IdeaPad Slim 5/82XD007HTW/i7...,https://www.momoshop.com.tw/goods/GoodsDetail....,2023-12-08,"$24,990(售價已折)",滿1件折1000,2023-12-12,"$24,990(售價已折)",滿1件折1000,2023-12-14,"$24,990(售價已折)",滿1件折1000
3,【Lenovo】15.6吋i5 RTX2050電競筆電(LOQ/i5-12450H/8G/5...,https://www.momoshop.com.tw/goods/GoodsDetail....,2023-12-08,"$21,990(售價已折)",滿1件折4000,2023-12-12,"$21,990(售價已折)",滿1件折4000,2023-12-14,"$21,990(售價已折)",滿1件折4000
4,【Lenovo】升級16G記憶體★15.6吋i7獨顯電競筆電(Gaming 3i/82S90...,https://www.momoshop.com.tw/goods/GoodsDetail....,2023-12-08,"$27,490(售價已折)",滿1件折9500,2023-12-12,"$27,290(售價已折)",滿1件折9700,2023-12-14,"$27,290(售價已折)",滿1件折9700
...,...,...,...,...,...,...,...,...,...,...,...
214,【ThinkPad 聯想】福利品 14吋i5商務筆電(E14/i5-1240P/8G/512...,https://www.momoshop.com.tw/goods/GoodsDetail....,NaN,NaN,NaN,2023-12-12,"$21,900",超值福利品!!,2023-12-14,"$21,900",超值福利品!!
215,【ThinkPad 聯想】升級24G組★E15 15.6吋商務筆電(i5-1240P/8G/...,https://www.momoshop.com.tw/goods/GoodsDetail....,NaN,NaN,NaN,2023-12-12,"$20,490(售價已折)",滿1件折11500,NaN,NaN,NaN
216,【ThinkPad 聯想】福利品 15.6吋i7商務筆電(E15/i7-1260P/8G/5...,https://www.momoshop.com.tw/goods/GoodsDetail....,NaN,NaN,NaN,2023-12-12,"$23,900(售價已折)",滿1件折1000,2023-12-14,"$23,900(售價已折)",滿1件折1000
217,【ThinkPad 聯想】+8G記憶體組★15.6吋i5商務筆電(E15/i5-1240P/...,https://www.momoshop.com.tw/goods/GoodsDetail....,NaN,NaN,NaN,2023-12-12,"$31,490(售價已折)",滿1件折500,2023-12-14,"$31,490(售價已折)",滿1件折500


# pivot table

In [68]:
# merged_df['Price'] = pd.to_numeric(merged_df['Price'].str.replace('[^\d]', '', regex=True), errors='coerce')

# 將 'Price' 列切割成指定的區間
bins = [0, 30000, 50000, 70000, 100000, float('inf')]
labels = ['3萬以下', '3~5萬', '5~7萬', '7~10萬', '10萬以上']

merged_df['Price_range'] = pd.cut(merged_df['Price'], bins=bins, labels=labels)

# 依照 'Price_range' 分組，並將每個價格區間的產品名稱轉換為字串列表
result = merged_df.groupby('Price_range')['product Name'].apply(list).reset_index()


In [72]:
result

,Price_range,product Name
0,3萬以下,[【Lenovo】15.6吋i5輕薄筆電(IdeaPad Slim 3/83EM0008TW...
1,3~5萬,[【ThinkPad 聯想】14吋i5商用獨顯筆電(T14/i5-1240P/16G/512...
2,5~7萬,[【Lenovo】16吋i7 RTX4070電競筆電(Legion Pro 5i/I7-13...
3,7~10萬,[]
4,10萬以上,[]
